In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import pandas as pd
import re

Read in prompts

In [3]:
df_work = pd.read_csv("/kaggle/input/falcon-wip/filtered_runs46_100.csv")
df_middle = pd.read_csv("/kaggle/input/falcon-wip/falcon_results_clean_run13_5.csv")
df_prompts = pd.read_csv("/kaggle/input/prompts-all-llms/prompts.csv")
df = df_prompts.copy()

Loading and initializing quantized Falcon 7b instruct model

In [4]:
model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

2024-03-07 19:48:23.609149: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 19:48:23.609280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 19:48:23.779501: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [6]:
# initialize empty lists to store generated texts for each run
generated_texts = [[] for _ in range(0,101)]

for run in range(0, 101):
    for index, row in df.iterrows():
        # obtain prompt for current row
        prompt = row['Prompt']

        # generate text
        sequences = pipeline(
            prompt,
            max_length=500,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )

        # extract the generated text from the sequences
        for seq in sequences:
            generated_text = seq['generated_text']
            
            generated_texts[run].append(generated_text)

# add each list of generated texts to df as new column
for run in range(0, 101):
    df[f'run_{run+1}'] = generated_texts[run]
    df.to_csv("working.csv")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Settin

In [7]:
# save initial results to csv
df_prompts.to_csv("falcon_results.csv")

I noticed that after we had changed our prompts, Mistral was performing very well and Falcon struggeled to return any type of answer and often just gave a rephrased version of our prompt rather than a proper response. I thus pre-filtered every cell of my results-dataframe and deleted any type of string matching our initial prompt. The empty cells are then filled by iterating over the dataframe and checking, if new text has to be generated.

Upon observing the count of empty cells, I realized that about 50% of responses after every Falcon 7B run were 'empty'. I stopped iterating, once I obtained valid responses for all cells.

In [8]:
def filter_responses(df, prompt_column, response_columns):
    for col in response_columns:
        filtered_responses = []
        for idx, row in df.iterrows():
            prompt = row[prompt_column]
            response = row[col]
            # Using regular expressions to match the start of the response with the prompt
            match = re.match(r'^{}[\s:]*'.format(re.escape(prompt)), response)
            if match:
                filtered_responses.append(response[match.end():].strip())
            else:
                filtered_responses.append(response)  # If no match, keep the original response
        df['filtered_' + col] = filtered_responses
    
    return df

In [11]:
# obtain list of column names for filtering - note that I used some test runs to check my results
col_strings_list = ['run_' + str(i) for i in range(6, 46)]
col_strings_list_long = ['run_' + str(i) for i in range(46, 101)]
col_strings_short = ['run_' + str(i) for i in range(1, 6)]
col_strings_full = ['run_' + str(i) for i in range (1, 101)]

In [12]:
# filter original data
#df_middle = filter_responses(df_middle, 'Prompt', col_strings_list)
#df_sample = filter_responses(df_sample, 'Prompt', col_strings_list_long)
#df_prompts = filter_responses(df_prompts, 'Prompt', col_strings_short)
df = filter_responses(df, 'Prompt', col_strings_full)

# save filtered version to csv file
#df_middle.to_csv('filtered_responses_6till45.csv', index=False)
#df_prompts.to_csv('filtered_runs_5_46.csv', index=False)
df.to_csv('filtered_runs.csv', index=False)

In [33]:
for idx, row in df.iterrows():
    for col_name in col_strings_full:
        filtered_col_name = 'filtered_' + col_name
        if pd.isna(row[filtered_col_name]) or row[filtered_col_name] == '':
            prompt = row['Prompt']
            
            # generate answer using the prompt
            sequences = pipeline(
                prompt,
                max_length=500,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
            )

            # obtain answer from sequences and filter out prompt
            for seq in sequences:
                generated_text = seq['generated_text']
                
            if generated_text:
                # update the corresponding cell with newly generated text
                df.loc[idx, col_name] = generated_text
            else:
                # handle exception when generated text is empty
                print("Generated text is empty for column:", col_name)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [16]:
df_prompts.to_csv("Falcon_working_version1.csv")

and now we start to clean and to iterate and do it over and over again... :)

In [34]:
# filter original data
df = filter_responses(df, 'Prompt', col_strings_full)

# save filtered version to csv file
df.to_csv('filtered_working_version1.csv', index=False)

In [35]:
# count empty strings and NaNs in dataframe
empty_string_count = (df == '').sum().sum()
print("Count of empty strings in the DataFrame:", empty_string_count)

df.isna().sum().sum()

Count of empty strings in the DataFrame: 0


0

In [87]:
df.to_csv('falcon_final.csv')